# Model Training (Deep Learning Models)

In this notebook, we develop and evaluate two deep learning models:
- **Artificial Neural Network (ANN)**
- **Convolutional Neural Network (CNN)**

Each model is trained and evaluated on:  
1️⃣ Feature Set 1 → Top 7 from each scale (21 features)  
2️⃣ Feature Set 2 → All PSS-10 + All PHQ-9 (19 features)  
3️⃣ Feature Set 3 → All GAD-7 + All PHQ-9 (17 features)

For each model and feature set, we show:
- Accuracy  
- Precision  
- Recall  
- F1 Score  
- Accuracy vs Epoch graph  
- Confusion Matrix

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import joblib
import tensorflow as tf
from tensorflow.keras import models, layers
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.utils import to_categorical

# Paths
DATA_DIR = Path("../data/processed")
MODEL_DIR = Path("../models")
FIG_DIR = Path("../figures")

MODEL_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

# Load feature sets
fs1_train = pd.read_csv(DATA_DIR / "fs1_train.csv")
fs1_test  = pd.read_csv(DATA_DIR / "fs1_test.csv")
fs2_train = pd.read_csv(DATA_DIR / "fs2_train.csv")
fs2_test  = pd.read_csv(DATA_DIR / "fs2_test.csv")
fs3_train = pd.read_csv(DATA_DIR / "fs3_train.csv")
fs3_test  = pd.read_csv(DATA_DIR / "fs3_test.csv")

print("✅ Feature sets loaded successfully!")

## Helper Functions for Building, Evaluating, and Saving Deep Learning Models

Includes:
- ANN and CNN architectures  
- Evaluation (Accuracy, Precision, Recall, F1, Confusion Matrix)  
- Plotting Accuracy vs Epoch  
- Saving all outputs

In [ ]:
def build_ann(input_dim, num_classes):
    """Simple feed-forward ANN model."""
    model = models.Sequential([
        layers.Dense(128, activation='relu', input_dim=input_dim),
        layers.Dropout(0.3),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


def build_cnn(input_dim, num_classes):
    """1D CNN adapted for tabular data."""
    model = models.Sequential([
        layers.Reshape((input_dim, 1), input_shape=(input_dim,)),
        layers.Conv1D(64, kernel_size=3, activation='relu'),
        layers.MaxPooling1D(pool_size=2),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


def evaluate_dl_model(model, X_test, y_test, feature_set_name, model_name):
    """Evaluate a DL model, compute metrics, and save confusion matrix."""
    y_pred_prob = model.predict(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1)
    y_true = np.argmax(y_test, axis=1)

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average="weighted", zero_division=0)
    rec = recall_score(y_true, y_pred, average="weighted", zero_division=0)
    f1 = f1_score(y_true, y_pred, average="weighted", zero_division=0)

    # --- Confusion Matrix ---
    cm = confusion_matrix(y_true, y_pred)
    fig, ax = plt.subplots(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
    ax.set_xlabel("Predicted Label")
    ax.set_ylabel("True Label")
    ax.set_title(f"Confusion Matrix — {model_name} ({feature_set_name})")
    fig_path = FIG_DIR / f"confusion_{feature_set_name.lower()}_{model_name.lower().replace(' ', '_')}.png"
    fig.savefig(fig_path, bbox_inches="tight", dpi=300)
    plt.close(fig)
    print(f"🖼️ Saved confusion matrix: {fig_path}")

    print(f"\n📊 {model_name} ({feature_set_name}) Performance:")
    print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1 Score: {f1:.4f}")

    return {"Model": model_name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1 Score": f1}


def plot_accuracy(history, feature_set_name, model_name):
    """Plot and save Accuracy vs Epoch."""
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.plot(history.history["accuracy"], label="Train Accuracy")
    ax.plot(history.history["val_accuracy"], label="Validation Accuracy")
    ax.set_title(f"Accuracy vs Epoch — {model_name} ({feature_set_name})")
    ax.set_xlabel("Epochs")
    ax.set_ylabel("Accuracy")
    ax.legend()
    fig_path = FIG_DIR / f"accuracy_{feature_set_name.lower()}_{model_name.lower().replace(' ', '_')}.png"
    fig.savefig(fig_path, bbox_inches="tight", dpi=300)
    plt.close(fig)
    print(f"📈 Saved accuracy vs epoch plot: {fig_path}")

## Model Training & Evaluation Function for ANN and CNN

In [ ]:
def train_dl_models(fs_name, train_df, test_df, epochs=50, batch_size=32):
    """Train and evaluate ANN & CNN on a given feature set."""
    X_train = train_df.drop(columns=["DepressionEncoded"]).values
    y_train = to_categorical(train_df["DepressionEncoded"])
    X_test = test_df.drop(columns=["DepressionEncoded"]).values
    y_test = to_categorical(test_df["DepressionEncoded"])

    input_dim = X_train.shape[1]
    num_classes = y_train.shape[1]

    results = []

    # ---- ANN ----
    print(f"\n🧠 Training ANN on {fs_name} ...")
    ann = build_ann(input_dim, num_classes)
    history_ann = ann.fit(X_train, y_train, validation_split=0.2, epochs=epochs, batch_size=batch_size, verbose=0)
    results.append(evaluate_dl_model(ann, X_test, y_test, fs_name, "ANN"))
    plot_accuracy(history_ann, fs_name, "ANN")
    ann.save(MODEL_DIR / f"{fs_name.lower()}_ann_model.h5")
    print(f"💾 Saved ANN model for {fs_name}")

    # ---- CNN ----
    print(f"\n🧩 Training CNN on {fs_name} ...")
    cnn = build_cnn(input_dim, num_classes)
    history_cnn = cnn.fit(X_train, y_train, validation_split=0.2, epochs=epochs, batch_size=batch_size, verbose=0)
    results.append(evaluate_dl_model(cnn, X_test, y_test, fs_name, "CNN"))
    plot_accuracy(history_cnn, fs_name, "CNN")
    cnn.save(MODEL_DIR / f"{fs_name.lower()}_cnn_model.h5")
    print(f"💾 Saved CNN model for {fs_name}")

    return pd.DataFrame(results)

## Feature Set 1 — Deep Learning Model Training and Evaluation

In [ ]:
results_dl_fs1 = train_dl_models("FS1", fs1_train, fs1_test)
display(results_dl_fs1)

## Feature Set 2 — Deep Learning Model Training and Evaluation

In [ ]:
results_dl_fs2 = train_dl_models("FS2", fs2_train, fs2_test)
display(results_dl_fs2)

## Feature Set 3 — Deep Learning Model Training and Evaluation

In [ ]:
results_dl_fs3 = train_dl_models("FS3", fs3_train, fs3_test)
display(results_dl_fs3)

## Save Deep Learning Model Results
All evaluation tables are saved for future comparison with traditional ML models.

In [ ]:
results_dl_fs1.to_csv(DATA_DIR / "results_fs1_deep_learning.csv", index=False)
results_dl_fs2.to_csv(DATA_DIR / "results_fs2_deep_learning.csv", index=False)
results_dl_fs3.to_csv(DATA_DIR / "results_fs3_deep_learning.csv", index=False)

print("✅ Saved Deep Learning results to data/processed/")